In [1]:
project_id = 'elife-data-pipeline'
output_dataset = 'de_dev'
output_table_prefix = 'data_science_'

In [2]:
import logging
import sys

import pandas as pd

import data_science_pipeline.configure_warnings  # pylint: disable=unused-import

from data_science_pipeline.sql import get_sql
from data_science_pipeline.utils.bq import run_query_and_save_to_table, get_client
from data_science_pipeline.utils.jupyter import printmd, to_markdown_sql, read_big_query

In [3]:
query_name = 'disambiguated_editor_papers'
destination_table_name = ''.join([output_table_prefix, query_name])

In [4]:
logging.basicConfig(level='INFO', stream=sys.stdout)

In [5]:
print('processing %s' % query_name)
_sql = get_sql('%s.sql' % query_name).format(
    project=project_id,
    dataset=output_dataset
)
printmd(to_markdown_sql(_sql))
run_query_and_save_to_table(
    client=get_client(project_id=project_id),
    query=_sql,
    destination_dataset=output_dataset,
    destination_table_name=destination_table_name
)
print('done')

processing disambiguated_editor_papers


> ```sql
> -- Main features:
> --    - Disambiguates editor linked papers as far as possible
> --      Gives each editor paper match a priority
> --    - Select papers with increasing priority until at least a target number of papers is reached
> 
> WITH t_editor AS (
>   SELECT
>     editor.person_id,
>     editor.relevant_pubmed_ids
>   FROM `elife-data-pipeline.de_dev.data_science_editor_pubmed_links` AS editor
> ),
> 
> t_pubmed_id_with_priority_by_person_id AS (
>   SELECT DISTINCT editor.person_id, pmid, 1 AS priority
>   FROM t_editor AS editor
>   JOIN UNNEST(relevant_pubmed_ids) AS pmid
> 
>   UNION DISTINCT
> 
>   SELECT DISTINCT
>     person_id,
>     pmid,
>     CASE
>       WHEN has_matching_orcid THEN 1
>       WHEN has_matching_last_name AND has_matching_first_name AND (has_matching_affiliation OR has_matching_previous_affiliation) THEN 2
>       WHEN has_matching_last_name AND has_matching_first_name AND has_matching_postal_code THEN 3
>       WHEN has_matching_last_name AND has_matching_first_name AND has_matching_city THEN 4
>       WHEN has_matching_last_name AND has_matching_first_name AND has_matching_country THEN 5
>       WHEN has_matching_last_name AND has_matching_first_name_letter AND (has_matching_affiliation OR has_matching_previous_affiliation) THEN 6
>       ELSE 7
>     END AS priority
>   FROM `elife-data-pipeline.de_dev.data_science_disambiguated_editor_papers_details`
>   WHERE NOT has_mismatching_orcid
> ),
> 
> t_priority_count_by_person_id AS (
>   SELECT person_id, priority, COUNT(*) AS priority_count
>   FROM t_pubmed_id_with_priority_by_person_id 
>   GROUP BY person_id, priority
> ),
> 
> t_max_available_priority_by_person_id AS (
>   SELECT person_id, MAX(priority) AS max_available_priority
>   FROM t_priority_count_by_person_id 
>   GROUP BY person_id
> ),
> 
> t_priority_count_and_total_priority_count_by_person_id AS (
>   SELECT
>     current_counts.*,
>     (
>       priority_count
>       + (
>         SELECT COALESCE(SUM(priority_count), 0)
>         FROM t_priority_count_by_person_id AS higher_priority_counts
>         WHERE higher_priority_counts.person_id = current_counts.person_id
>         AND higher_priority_counts.priority < current_counts.priority
>       )
>     ) AS total_priority_count
>   FROM t_priority_count_by_person_id AS current_counts
> ),
> 
> t_max_preferred_priority_by_person_id AS (
>   SELECT
>     counts.person_id,
>     MIN(priority) AS max_preferred_priority
>   FROM t_priority_count_and_total_priority_count_by_person_id AS counts
>   JOIN t_max_available_priority_by_person_id AS max_available
>     ON max_available.person_id = counts.person_id
>   WHERE total_priority_count >= 50
>     OR priority = max_available.max_available_priority
>   GROUP BY counts.person_id
> ),
> 
> t_preferred_pubmed_id_by_person_id AS (
>   SELECT DISTINCT
>     max_preferred.person_id,
>     paper.pmid
>   FROM t_max_preferred_priority_by_person_id AS max_preferred
>   JOIN t_pubmed_id_with_priority_by_person_id AS paper
>     ON paper.person_id = max_preferred.person_id
>     AND paper.priority <= max_preferred.max_preferred_priority
>     AND paper.pmid IS NOT NULL
> )
> 
> SELECT
>   editor.person_id,
>   ARRAY(
>     SELECT pmid
>     FROM t_preferred_pubmed_id_by_person_id AS preferred_paper
>     WHERE preferred_paper.person_id = editor.person_id
>   ) AS disambiguated_pubmed_ids
> FROM t_editor AS editor
> ```

INFO:data_science_pipeline.utils.bq:ran query and saved to: de_dev.data_science_disambiguated_editor_papers, total rows: 634, took: 6.682s
done


In [6]:
_sql = get_sql('disambiguated_editor_papers_count.sql').format(
    project=project_id,
    dataset=output_dataset
)
with pd.option_context("display.max_rows", 1000):
    print(read_big_query(_sql).to_string(index=False))

> ```sql
> SELECT
>   Profile.Name AS name,
>   ARRAY_LENGTH(papers.disambiguated_pubmed_ids) AS pubmed_count
> FROM `elife-data-pipeline.de_dev.mv_Editorial_Editor_Profile` AS Profile
> LEFT JOIN `elife-data-pipeline.de_dev.data_science_disambiguated_editor_papers` AS papers
>   ON papers.person_id = Profile.Person_ID
> WHERE Profile.Name IS NOT NULL
> ORDER BY Profile.Name
> ```

Downloading: 100%|██████████| 633/633 [00:00<00:00, 745.02rows/s]

                             name  pubmed_count
                    Abby Dernburg           5.0
                        Adam Aron           0.0
                       Adam Frost           0.0
                    Adam Linstedt           0.0
                  Adrien Peyrache          27.0
                    Adèle Marston          38.0
                  Agnese Seminara           0.0
              Agnieszka Chacinska           5.0
                     Ahmet Yildiz          18.0
                  Alan Hinnebusch          10.0
                       Alan Moses           7.0
                   Albert Cardona          19.0
       Alejandro Sánchez Alvarado          42.0
               Aleksandra Walczak          10.0
                        Alex Cook           4.0
                     Alex Fornito           5.0
                  Alexander Borst           4.0
               Alexander Shackman           8.0
             Alexander Westermann           6.0
                 Alfonso Valencia       